In [59]:
import random
import os
import pandas as pd
import numpy as np
import copy
import operator

In [ ]:
# словарь отношений я создал на предыдущей стадии.

In [ ]:
# импортирую словарь отношений. Он формата {'человек':{'другой человек':коэффициент, и т.д.}, и т.д.}

In [121]:
with open('full_dict_relationsRazval4.txt', 'r') as f:
    full_dict_relations = eval(f.read())

In [128]:
# reset
list_groups = [[],[],[],[],[],[]]
last_people_base = list(full_dict_relations.keys())
last_people = list(full_dict_relations.keys())

In [80]:
# принимает номер группы, список групп. remove отвечает за удаление выбранного человека из данного списка групп. Возвращает имя идеального для группы человека
def best_person(group, last_people, remove = True):
    mid_dict = {}
    for i in last_people:
        sum_1_to_others = 0
        for j in list_groups[group - 1]:
            try:
                sum_1_to_others += full_dict_relations[i][j]
            except:
                pass
            try:
                sum_1_to_others += full_dict_relations[j][i]
            except:
                pass
        mid_dict[i] = sum_1_to_others
    sorted_dict = {k: v for k, v in sorted(mid_dict.items(), key=lambda item: item[1], reverse = True)}
    a = list(sorted_dict.keys())[0]
    if remove:
        last_people.remove(a)
    return a

In [81]:
# получает номер группы, возвращает сумму всех отношений
def sum_relation(group, list_groups):
    all_sum = 0
    for i in list_groups[group - 1]:
        for j in list_groups[group - 1]:
            try:
                all_sum += full_dict_relations[i][j]
            except:
                pass
            try:
                all_sum += full_dict_relations[j][i]
            except:
                pass
    return all_sum

In [82]:
# получает список групп, возвращает общую сумму и численность всех групп в сумме как tuple()
def sum_all_groups(list_groups):
    all_sum = 0
    population = 0
    for i in range(1, 7):
        population += len(list_groups[i - 1])
        all_sum += sum_relation(i, list_groups)
    return all_sum, population

In [83]:
# получает ключ человека в словаре отношений и номер группы.
# возвращает человека и его вклад в группу.
def contribution_one_person(person, num_group, list_groups):
    sum_contr = 0
    for i in list_groups[num_group - 1]:
        try:
            sum_contr += full_dict_relations[person][i]
        except:
            pass
        try:
            sum_contr += full_dict_relations[i][person]
        except:
            pass
    return (person, sum_contr)

In [646]:
# найти шесть лучших пятнадцаток.
# 6 групп, которые заполнены по 15 человек с максимальными суммами.

In [129]:
for i in range(6):
    dict_10_groups = {}
    last_10_people = {}
    for k in last_people_base[:]:     
        list_groups[i] = []
        last_people = last_people_base[:]
        list_groups[i].append(k)
        last_people.remove(k)
        for j in range(14):  # если 14, тогда будет по 15 человек в группах.
            list_groups[i].append(best_person(i + 1, last_people, remove = True))
        a = sum_relation(i + 1, list_groups = list_groups)
        last_10_people[a] = last_people
        dict_10_groups[a] = list_groups[i]
    list_groups[i] = dict_10_groups[max(dict_10_groups)]
    last_people = last_10_people[max(dict_10_groups)]
    for people in list_groups[i]:
        last_people_base.remove(people)

In [124]:
#здесь можно проверить общую сумму, численность и оставшихся людей
sum_all_groups(list_groups = list_groups)

(11718.399999999969, 90)

In [125]:
len(last_people)

69

In [5]:
# их заполнить с перетасовкой

In [65]:
# дать группу человека, его имя и коэф для тюна для толерантности к изменению его вклада в группу. Возвращает список рациональных переходов для этого человека.
def allowed_groups(group, name, coeff_min):
    _, current_contr = contribution_one_person(name, group, list_groups)
    minim_contrib = current_contr / coeff_min
    list_allowed_groups = []
    for i in range(6):
        _, contr = contribution_one_person(name, i + 1, list_groups)
        if contr > minim_contrib or current_contr == 0:
            list_allowed_groups.append(i + 1)
    if group not in list_allowed_groups:
        list_allowed_groups.append(group)
    return list_allowed_groups

In [66]:
# дать словарь возможных комбинаций. Возвращает количество комбинаций.
def num_comb(dictionary):
    list_comb = []
    n_comb = 1
    for key, value in dictionary.items():
        n_comb = n_comb * len(value)
    return n_comb

In [67]:
# дать словарь с возможными комбинациями и список групп. Он выдаст новую комбинацию в виде нового списка групп и количество комбинаций.
def best_combination(dictionary, groups):
    n_comb = num_comb(dictionary)
    dict_comb = {}
# Здесь реализован перебор всех вариантов. 
    help_dict = {}
    new_comb = n_comb
    for key, value in dictionary.items():
        if len(value) > 1:
            new_comb = int(new_comb / len(value))
            help_dict[key] = new_comb
    for i in range(n_comb):
        ground_list_groups = copy.deepcopy(groups)
        for key, value in dictionary.items():
            if len(value) > 1:
                number = (i // help_dict[key]) % len(value)
                value_num = value[number]
                ground_list_groups[value_num - 1].append(key)
            else:
                ground_list_groups[value[0] - 1].append(key)
        sum_gr, _ = sum_all_groups(ground_list_groups)
        dict_comb[str(ground_list_groups)] = sum_gr
#         данный код сбрасывает лист после 10000 комбинаций, оставляя лучший вариант, чтобы разгружать ОЗУ.
        if (i + 1) % 10000 == 0:
            print(f'{(i + 1) // 10000} iteration with {n_comb} combinations')
            dict_comb = strip_max(dict_comb)
            left_n_max(dict_comb, 1)
# если ключом оставлять сумму, тогда могут быть перезатирания, поэтому ключ - длинный список списков.
    dict_comb = strip_max(dict_comb)
    return eval(max(dict_comb, key=dict_comb.get)), n_comb

In [126]:
# удаляет все значения, где число людей в групп > 30. Дать словарь dict_comb из предыдущей функции.
def strip_max(dictionary):
    dict1 = copy.deepcopy(dictionary)
    for key, values in dictionary.items():
        for i in eval(key):
            if len(i) > 30:
                try:
                    del dict1[key]
                except:
                    pass
    return dict1

In [69]:
# оставляет в словаре n значений с наивысшими значениями. Тоже дать dict_comb. 
def left_n_max(d, n=1):
    for _ in range(len(d) - n):
        min_key = min(d.keys(), key=lambda k: d[k])
        del d[min_key]

In [70]:
# неактивен. Думал при большом количестве комбинаций уменьшать коэффициенты и глубину перетасовки.
def reorder(dict_min, list_groups, depth, coeff_min):
    for key, value in dict_min.items():
        for j in value:
            list_groups[key - 1].append(j)
    if depth > 1:
        depth = depth - 1
    else:
        coeff_min = coeff_min * 0.8
    dict_min = {}
    for group in range(6):
        dict_rel = {}
        for i in list_groups[group]:
            name, contr = contribution_one_person(i, group + 1, list_groups)
            dict_rel[name] = contr
        list_min = []
        for i in sorted(dict_rel.items(), key=lambda item: item[1])[:depth]:
            list_min.append(i[0])
        dict_min[group + 1] = list_min
        for i in list_min:
            list_groups[group].remove(i)
    dict_change_groups = {}
    for key, value in dict_min.items():
        for i in value:
            dict_change_groups[i] = allowed_groups(key, i, coeff_min)
    dict_change_groups[free] = list(range(1, 7))
    return dict_change_groups

In [ ]:
# пытался это прикрутить, чтобы группы заполнялись более равномерно.
# max([len(x) for x in i]) - min([len(x) for x in i]) > 5

In [130]:
# depth - сколько мы забираем людей из каждой группы при каждой итерации
# coeff_min - насколько мы готовы уменшить вклад человека в группу, чтобы посчитать его переход в другую группу хоть частично разумным
depth = 3
coeff_min = 1
# n_group_for_free = 2 # использовал для того, чтобы свободному человеку давать не все, а три лучшие группы и первую минимальную.

In [116]:
# делаю излишние копии, потому что у меня уже сдвиг на связанных списках.

In [131]:
while len(last_people) > 0:
# следущий код находит по {depth} людей, которые меньше всего значат для группы. И их удаляет и добавляет в словарь dict_change_groups, откуда она будут рассортированы
    dict_min = {}
    for group in range(6):
        dict_rel = {}
        for i in list_groups[group]:
            name, contr = contribution_one_person(i, group + 1, list_groups)
            dict_rel[name] = contr
        list_min = []
        for i in sorted(dict_rel.items(), key=lambda item: item[1])[:depth]:
            list_min.append(i[0])
        dict_min[group + 1] = list_min
        for i in list_min:
            list_groups[group].remove(i)
# для этих удалённых людей этот код создаёт словарь с возможными их переходами в другие группы.
    dict_change_groups = {}
    for key, value in dict_min.items():
        for i in value:
            dict_change_groups[i] = allowed_groups(key, i, coeff_min)
# а этот даёт новому человеку все 6 групп как возможные.
    free = last_people.pop()
    dict_change_groups[free] = list(range(1, 7))
# этот находит новую комбинацию. Далее собираются значения и выводятся на экран.
    new_list_groups, n_comb = copy.deepcopy(best_combination(dict_change_groups, copy.deepcopy(list_groups)))
    all_sum, pop = sum_all_groups(list_groups = new_list_groups)
    list_groups = copy.deepcopy(new_list_groups)
    
    for j, i in enumerate(list_groups):
        print(f"{len(i)} - {round(sum_relation(j, list_groups), 2)}|", end = '')
    print(f'{round(all_sum, 1)} {pop} {n_comb} {round(all_sum / pop, 2)}')  

15 - 1252.6|15 - 2283.8|16 - 2182.2|15 - 2197.0|15 - 2131.4|15 - 1693.4|11740.4 91 6 129.02
15 - 1252.6|15 - 2283.8|17 - 2182.2|15 - 2235.6|15 - 2131.4|15 - 1693.4|11779.0 92 6 128.03
15 - 1252.6|15 - 2283.8|17 - 2182.2|16 - 2235.6|15 - 2167.4|15 - 1693.4|11815.0 93 6 127.04
15 - 1252.6|15 - 2283.8|17 - 2182.2|17 - 2235.6|15 - 2209.4|15 - 1693.4|11857.0 94 6 126.14
15 - 1252.6|16 - 2283.8|17 - 2276.2|17 - 2235.6|15 - 2209.4|15 - 1693.4|11951.0 95 6 125.8
15 - 1252.6|16 - 2283.8|17 - 2276.2|17 - 2235.6|16 - 2209.4|15 - 1733.4|11991.0 96 6 124.91
15 - 1252.6|16 - 2283.8|17 - 2276.2|17 - 2235.6|17 - 2209.4|15 - 1755.4|12013.0 97 6 123.85
15 - 1252.6|16 - 2283.8|18 - 2276.2|17 - 2244.6|17 - 2209.4|15 - 1755.4|12022.0 98 36 122.67
15 - 1252.6|16 - 2283.8|17 - 2276.2|17 - 2235.6|19 - 2209.4|15 - 1834.4|12092.0 99 216 122.14
15 - 1252.6|16 - 2283.8|17 - 2276.2|17 - 2235.6|20 - 2209.4|15 - 1969.0|12226.6 100 6 122.27
15 - 1252.6|16 - 2283.8|17 - 2276.2|17 - 2235.6|21 - 2209.4|15 - 2059.4|12317

In [100]:
for i in list_groups:
    print(len(i))

29
30
23
30
30
17


In [ ]:
# далее я сохраняю список. И также выгружаю list_groups, т.е. готовый файл в .xlsx

In [74]:
with open('list_groupsRazval.txt', 'w') as f:
    f.write(str(list_groups))

In [132]:
excel_groups = pd.DataFrame({'num':range(1, 31)})

In [133]:
for num, i in enumerate(list_groups):
    excel_groups[f'{num + 1} group'] = pd.Series(i)

In [134]:
excel_groups.to_excel('Razval3.xlsx', index=None)